# 📝 Steps for Working with `retail_store_inventory.csv`

In this project, we need to explore, clean, and prepare the raw sales and inventory data before feeding it into forecasting and simulation models. Here are the detailed steps:

---

## 1. Initial Data Exploration
- Check dataset shape (rows × columns).
- Preview first few rows (`head`).
- Inspect data types and missing values (`info`, `isnull().sum()`).
- Generate summary statistics (`describe`).

---
wwdawdas


   
## 2. Identify Key Columns
Expected columns are:
- `date` → date of the record  
- `store_id` → store/branch identifier  
- `product_id` → product identifier  
- `sales_units` → daily sales units  
- `inventory_level` → daily stock level  
- `price` → product price (if available)  
- `promotion` or other contextual features (optional)

---

## 3. Handle Missing Values
- Detect missing values in each column.  
- Decide how to fill:
  - Replace with **0** if missing means no sales/stock.  
  - Use **interpolation** if values are actually missing in sequence.  
  - Drop rows if the missingness is negligible.  

---

## 4. Convert Data Types
- Convert `date` → **datetime**.  
- Ensure numerical fields (`sales_units`, `inventory_level`, `price`) are numeric.  
- Convert categorical fields (`store_id`, `product_id`) to **category** type.  

---

## 5. Check Uniqueness
- How many unique products?  
- How many unique stores?  
- This helps narrow down the scope (e.g., focus on 2–3 products and 1–2 stores for the MVP).  

---

## 6. Time Series Analysis
- Plot total daily sales to inspect overall trend.  
- Plot per product/store to detect seasonality or weekly patterns.  
- Look for trend, seasonality, and noise in the data.  

---

## 7. Distribution Analysis
- Plot histogram of `sales_units` to check sales distribution.  
- Plot histogram of `inventory_level` to assess stock variability.  
- Identify potential outliers.  

---

## 8. Feature Relationships
- Correlation between `price` and `sales_units` → price elasticity.  
- Effect of `promotion` on sales → uplift estimation.  
- If features are missing, create synthetic assumptions for simulation.  

---

## 9. Data Preparation for Modeling
Restructure data into standard tables:
- **sales.csv** → [date, product_id, store_id, units, revenue]  
- **inventory.csv** → [date, product_id, store_id, stock_level]  
- **products.csv** → [product_id, name?, price, elasticity?, reorder_point, lead_time]  
- **branches.csv** → [store_id, city?]  

---

✅ After these steps, the dataset will be clean and structured, ready for **Forecasting → Simulation → Agentic AI integration**.


In [1]:
import sys
import torch
import timesfm
import numpy as np
import pandas as pd
import torch.nn as nn
from pathlib import Path
import torch.optim as optim
import matplotlib.pyplot as plt
from timesfm import ForecastConfig
from sklearn.metrics import mean_squared_error
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import  StandardScaler
from sklearn.model_selection import train_test_split


In [2]:
print("PY:", sys.executable)
print("timesfm from:", timesfm.__file__)
print("has TimesFm?", hasattr(timesfm, "TimesFm"))
print("has old class?", hasattr(timesfm, "TimesFM_2p5_200M_torch"))

PY: c:\Users\Taha\AppData\Local\Programs\Python\Python312\python.exe
timesfm from: C:\Users\Taha\OneDrive\Desktop\OpenEdge-Project\timesfm\src\timesfm\__init__.py
has TimesFm? False
has old class? True


### 0) setting input and output paths :

SRC means source path of the input file


OUT_DIR means output directory where processed files will be saved

  . path means a filesystem path

  . out_dir.mkdir(parents=True, exist_ok=True) means create the output directory if it doesn't exist already


In [3]:
SRC  = r"C:\Users\Taha\OneDrive\Desktop\OpenEdge-Project\retail_store_inventory.csv"
OUT_DIR = Path("./data/processed")

# parent=True allows creation of parent directories if they don't exist
# exist_ok=True prevents error if the directory already exists
OUT_DIR.mkdir(parents=True, exist_ok=True) # .mkdir is used to create directory if it doesn't exist
OUT_STD = OUT_DIR / "standardized_raw.csv" 

df = pd.read_csv(SRC)

In [4]:
for col in df.columns:
    print(col)

Date
Store ID
Product ID
Category
Region
Inventory Level
Units Sold
Units Ordered
Demand Forecast
Price
Discount
Weather Condition
Holiday/Promotion
Competitor Pricing
Seasonality


# 2) rename به snake_case 
    . snake_case means lowercase with underscores instead of spaces or camelCase
    . rename columns to snake_case for consistency and easier access    

In [5]:
rename_map = {
    "Date": "date", #date means date of transaction
    "Store ID": "store_id", #store_id means unique identifier for each store
    "Product ID": "product_id", #product_id means unique identifier for each product
    "Category": "category", #category means category of the product
    "Region": "region", #region means geographical region of the store
    "Inventory Level": "inventory_level", #inventory_level means current stock level of the product in the store
    "Units Sold": "units_sold", #units_sold means number of units sold in the transaction
                                #the total number of products sold by a company during a specific period
    "Units Ordered": "units_ordered", #units_ordered means number of units ordered from suppliers
    "Demand Forecast": "demand_forecast", #demand_forecast means predicted demand for the product
    "Price": "price", 
    "Discount": "discount",
    "Weather Condition": "weather",
    "Holiday/Promotion": "holiday_promo", #holiday_promo means whether the transaction occurred during a holiday or promotion period
    "Supplier Lead Time": "supplier_lead_time", #supplier_lead_time means time taken by supplier to deliver the product
    "Competitor Pricing": "competitor_price", #competitor_price means price of similar products from competitors
    "Seasonality": "seasonality", #seasonality means seasonal trend affecting the product sales
}
df = df.rename(columns=rename_map)

# print(df["holiday_promo"].value_counts()) # Check unique values in 'holiday_promo' column
# print(df["weather"].value_counts()) # Check unique values in 'weather' column
# print(df["seasonality"].value_counts()) # Check unique values in 'seasonality' column
# print(df["category"].value_counts()) # Check unique values in 'category' column
# print(df["region"].value_counts()) # Check unique values in 'region' column
# print(df["date"].min(), df["date"].max()) # Check date range
# print(df.isnull().sum()) # Check for missing values
# print(df.dtypes) # Check data types of each column



# 3) casting |>
     .3.1 datetime

In [6]:
if "date" in df.columns:
    df["date"] = pd.to_datetime(df["date"], errors="coerce") # Convert to datetime, coerce errors to NaT

     .3.2 category/string

In [7]:
for c in ["store_id", "product_id", "category", "region"]:
    if c in df.columns:
        df[c] = df[c].astype("string")  # Convert to string

     .3.2 numeric/stri

In [8]:
for c in [
    "inventory_level", "units_sold", "units_ordered", "demand_forecast",
    "price", "discount", "competitor_price", "seasonality"
]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")  # Convert to numeric, coerce errors to NaN

In [9]:
if "holiday_promo" in df.columns:
    df["holiday_promo"] = (
        df["holiday_promo"]
        .astype("string")
        .str.lower()
        .map({
            "yes": 1, "y": 1, "1": 1,
            "no": 0, "n": 0, "0": 0
        })
        .fillna(0)   # treat NaN as no
        .astype("int8")
    )

if "weather" in df.columns:
    df["weather"] = (
        df["weather"]
        .astype("string")
        .str.lower()
        .map({
            "sunny": 0,
            "cloudy": 1,
            "rainy": 2,
            "snowy": 3
        })
        .fillna(0)  # treat NaN as sunny
        .astype("int8")
    )

# aggregation rules
    .  sales_units = sum of sales_units per day per product per store
    .  inventory_level = last recorded inventory_level per day per product per store
    .  price = average price per day per product per store
    .  discount = average discount per day per product per store
    .  weather = most frequent weather condition per day per product per store
    .  holiday_promo = most frequent holiday_promo status per day per product per store
    .  seasonality = most frequent seasonality status per day per product per store
    .  competitor_price = average competitor_price per day per product per store

In [10]:
agg_rules = {
    "unit_sold": "sum", # total units sold in the day
    "inventory_level": "last", # inventory level at the end of the day
    'price': "mean", # average price of the product during the day
    'discount': "max", # maximum discount offered during the day
    "competitor_price": "mean", # average competitor price during the day
    "units_ordered": "sum", # total units ordered from suppliers during the day
    "demand_forecast": "mean", # means average predicted demand for the product
    "holiday_promo": "max", # if any transaction during holiday/promo, mark as 1
    "weather": "last", # worst weather condition during the day
    "category": "last", # category of the product
    "region": "last", # region of the store
    "seasonality": "last" # seasonal trend affecting the product sales
}

# Determine key columns for grouping. It becauses dynamic to avoid KeyError if any column is missing.
key_cols = [c for c in ["date", "store_id", "product_id"] if c in df.columns]
print("Key columns for grouping:", key_cols)
print()
value_cols = [c for c in agg_rules.keys() if c in df.columns and c not in key_cols]
print("Value columns for aggregation:", value_cols)
"""
 --- Grouping and Aggregation ---
1) Group the data by the key columns (date, store_id, product_id).
    We use dropna=False so that rows with NaN in key columns are still included as separate groups.
2) Select only the value columns we want to aggregate (e.g., units_sold, price, discount, etc.).
3) Apply the aggregation rules defined in agg_rules:
    - sum for total quantities (e.g., units_sold, units_ordered)
    - mean for average values (e.g., price, competitor_price, demand_forecast)
    - last for end-of-day or categorical values (e.g., inventory_level, weather, category, region)
    - max for flags or discounts (e.g., discount, holiday_promo)
 4) Reset the index so the grouped keys become normal columns again.
"""
if set(key_cols) == {"date","store_id","product_id"}: # Ensure all key columns are present
    df = (
        df.groupby(key_cols, dropna=False)[value_cols] # dropna=False to include NaN groups
                                                       # groupby means to group data based on key columns
          .agg({c: agg_rules[c] for c in value_cols}) # agg means to aggregate data based on agg_rules
          .reset_index()
    )
else:
    print("date, store_id, product_id")
    

Key columns for grouping: ['date', 'store_id', 'product_id']

Value columns for aggregation: ['inventory_level', 'price', 'discount', 'competitor_price', 'units_ordered', 'demand_forecast', 'holiday_promo', 'weather', 'category', 'region', 'seasonality']


In [11]:
print("✅ Done standardization\n")
print("Shape:", df.shape)
print("\nDate range:", df["date"].min(), "→", df["date"].max() if "date" in df.columns else "(no date)")
print("\nNull counts:\n", df.isnull().sum())

# Check for duplicates based on key columns
# all checks if all key columns are present in the dataframe
# duplicated counts how many duplicate rows exist based on the subset of key columns
# there are other methods except all like any, sum, etc.
if all(k in df.columns for k in ["date","store_id","product_id"]): # Ensure all key columns are present
    dups = df.duplicated(subset=["date","store_id","product_id"]).sum() # Count duplicates based on key columns
    print("\nDuplicates by (date, store_id, product_id):", dups)

display(df.head(10))

df.to_csv(OUT_STD, index=False)
print(f"\n💾 Saved standardized file → {OUT_STD.resolve()}")

✅ Done standardization

Shape: (73100, 14)

Date range: 2022-01-01 00:00:00 → 2024-01-01 00:00:00

Null counts:
 date                    0
store_id                0
product_id              0
inventory_level         0
price                   0
discount                0
competitor_price        0
units_ordered           0
demand_forecast         0
holiday_promo           0
weather                 0
category                0
region                  0
seasonality         73100
dtype: int64

Duplicates by (date, store_id, product_id): 0


,date,store_id,product_id,inventory_level,price,discount,competitor_price,units_ordered,demand_forecast,holiday_promo,weather,category,region,seasonality
0,2022-01-01,S001,P0001,231,33.50,20,29.69,55,135.47,0,2,Groceries,North,NaN
1,2022-01-01,S001,P0002,204,63.01,20,66.16,66,144.04,0,0,Toys,South,NaN
2,2022-01-01,S001,P0003,102,27.99,10,31.32,51,74.02,1,0,Toys,West,NaN
3,2022-01-01,S001,P0004,469,32.72,10,34.74,164,62.18,1,1,Toys,North,NaN
4,2022-01-01,S001,P0005,166,73.64,0,68.95,135,9.26,0,0,Electronics,East,NaN
5,2022-01-01,S001,P0006,138,76.83,10,79.35,102,139.82,1,0,Groceries,South,NaN
6,2022-01-01,S001,P0007,359,34.16,10,36.55,167,108.92,1,2,Furniture,East,NaN
7,2022-01-01,S001,P0008,380,97.99,5,100.09,54,329.73,0,1,Clothing,North,NaN
8,2022-01-01,S001,P0009,183,20.74,10,17.66,135,174.15,0,1,Electronics,West,NaN
9,2022-01-01,S001,P0010,108,59.99,0,61.21,196,24.47,1,2,Toys,South,NaN



💾 Saved standardized file → C:\Users\Taha\OneDrive\Desktop\OpenEdge-Project\data\processed\standardized_raw.csv


#    Dataframe settings and secure copying

In [12]:
dfq = df.copy()

In [13]:
# مرتب‌سازی برای محاسبات شیفت
dfq = dfq.sort_values(["store_id","product_id","date"]).copy()

# اگر سفارش بعضی روزها NaN است، برای تخمین فروش 0 در نظر بگیر (می‌تونی سیاست دیگری انتخاب کنی)
dfq["units_ordered"] = pd.to_numeric(dfq["units_ordered"], errors="coerce").fillna(0)

# موجودی روز قبل در هر (store, product)
dfq["prev_inventory"] = dfq.groupby(["store_id","product_id"])["inventory_level"].shift(1)

# تخمین فروش: prev + ordered - current ، منفی‌ها را 0 کن
dfq["units_sold_est"] = (dfq["prev_inventory"] + dfq["units_ordered"] - dfq["inventory_level"]).clip(lower=0)

# اگر ستون units_sold وجود ندارد، همین تخمین را به‌عنوان ستون فروش اصلی بساز
if "units_sold" not in dfq.columns:
    dfq["units_sold"] = dfq["units_sold_est"]
else:
    # اگر وجود دارد، فقط جاهای خالی را با تخمین پر کن
    dfq["units_sold"] = pd.to_numeric(dfq["units_sold"], errors="coerce") # pd.to_numeric is used to convert a column to numeric type, coercing errors to NaN
    dfq["units_sold"] = dfq["units_sold"].fillna(dfq["units_sold_est"])

# (اختیاری) پرچم بده که کجاها از تخمین استفاده شد
dfq["units_sold_imputed_from_balance"] = (
    dfq["units_sold_est"].notna() & (("units_sold" not in dfq.columns) or dfq["units_sold"].isna())
).astype("int8")

print(
    "rows with units_sold_est > 0:", int((dfq["units_sold_est"]>0).sum()),
    "| final units_sold nulls:", int(dfq["units_sold"].isna().sum())
)


rows with units_sold_est > 0: 51466 | final units_sold nulls: 100


In [14]:
key_cols = [c for c in ["date", "store_id", "product_id"] if c in df.columns]
num_cols_missing_policy = ["inventory_level", "price", "competitor_price", "units_ordered", "units_sold"]
num_cols_outlier_policy = ["inventory_level", "price", "competitor_price", "units_ordered", "units_sold"]

# Missing Value Policy
MISSING_DROP_THRESHOLD = 0.01  # If more than 50% values are missing, drop the column

###     Flag Missing/Invalid and "Delete if low, Impute if high

In [15]:
# Basic assertions to ensure key columns and date validity
assert all(k in dfq.columns for k in key_cols), "Key columns are missing in the dataframe"
assert dfq["date"].notna().any(), "No valid dates"

# Create missing value flags for numerical columns
for col in num_cols_missing_policy:
    if col in dfq.columns:
        dfq[f"{col}_isna"] = dfq[col].isna().astype("int8")  # Flag missing values

# Calculate missing rates for numerical columns
missing_rates = {} # Dictionary to store missing rates
for col in num_cols_missing_policy:
    if col in dfq.columns:
        rate = dfq[f"{col}_isna"].mean() # Calculate missing rate
        missing_rates[col] = rate # [col] is the key, rate is the value

print("Missing rates:", {k: f"{v:.2%}" for k, v in missing_rates.items()})

# Decide which columns to drop or impute based on missing rates
to_drop_mask = pd.Series(False, index=dfq.columns) # Initialize mask to False for all columns

for col, rate in missing_rates.items():
    if rate <= MISSING_DROP_THRESHOLD:
        to_drop_mask = to_drop_mask | dfq[col].isna()
    else:
        dfq[f"needs_impute_{col}"] = dfq[col].isna().astype("int8")

# Drop rows with missing values in columns that are below the missing threshold
rows_before = len(dfq)
print(f"Rows before dropping: {rows_before}")

# ~ means NOT, so we keep rows where to_drop_mask is False. ~ changes True to False and False to True
# Reset index after dropping rows, drop=True avoids adding old index as a column
dfq = dfq.loc[~to_drop_mask].reset_index(drop=True)
rows_after = len(dfq)

print(f"Dropped {rows_before - rows_after} rows due to low missing rates policy.")


Missing rates: {'inventory_level': '0.00%', 'price': '0.00%', 'competitor_price': '0.00%', 'units_ordered': '0.00%', 'units_sold': '0.14%'}
Rows before dropping: 73100
Dropped 100 rows due to low missing rates policy.


### Winsorization with `groupby` + `apply`

We use `groupby` on `["store_id", "product_id"]` so that outlier detection is done **per store–product group** rather than across the whole dataset.  
The helper function `_wins` is applied to each group separately.

---

#### Example Data
| store_id | product_id | price |
|----------|------------|-------|
| S1       | P1         | 100   |
| S1       | P1         | 95    |
| S1       | P2         | 50    |
| S2       | P1         | 200   |
| S2       | P1         | 210   |

---

#### Step 1 — Grouping
Groups created by `["store_id","product_id"]`:
1. (S1,P1) → [100, 95]  
2. (S1,P2) → [50]  
3. (S2,P1) → [200, 210]  

---

#### Step 2 — Winsorization per group
- Compute **Q1, Q3, IQR** for the column (here: `price`) inside each group.  
- Define bounds:  
  - Lower Bound = Q1 − 1.5 × IQR  
  - Upper Bound = Q3 + 1.5 × IQR  
- Clip values outside `[low, high]` back into the range.  
- Mark rows that changed with an outlier flag.

---

#### Step 3 — Output
| store_id | product_id | price | price_outlier_flag | price_w | price_low_iqr | price_high_iqr |
|----------|------------|-------|---------------------|---------|---------------|----------------|
| S1       | P1         | 100   | 0                   | 100     | 87.5          | 107.5          |
| S1       | P1         | 95    | 0                   | 95      | 87.5          | 107.5          |
| S1       | P2         | 50    | 0                   | 50      | NaN           | NaN            |
| S2       | P1         | 200   | 0                   | 200     | 185.0         | 225.0          |
| S2       | P1         | 210   | 0                   | 210     | 185.0         | 225.0          |

---

#### Notes
- `price_outlier_flag = 1` means the value was outside the IQR bounds and got clipped.  
- `price_w` is the winsorized (clipped) version of `price`.  
- `price_low_iqr` / `price_high_iqr` store the thresholds used for each group.  
- In this example, no outliers were detected, so flags are 0 and clipped values equal original ones.


In [16]:
group_cols = ["store_id", "product_id"]

def winsorize_grouped(df_in, col, group_cols, k=1.5):
    """
    Winsorize a numerical column within each group to limit the effect of outliers.
    
    Parameters:
    - group: DataFrameGroupBy object
    - col: Column name to winsorize
    - lower_quantile: Lower quantile threshold (default 1%)
    - upper_quantile: Upper quantile threshold (default 99%)
    
    Returns:
    - Series with winsorized values

    1) Calculate Q1 (25th percentile) and Q3 (75th percentile) for the specified column within each group.
    2) Compute the IQR (Interquartile Range) as Q3 - Q1.
    3) Determine the lower and upper bounds for outliers using the formula:
         - Lower Bound = Q1 - k * IQR
         - Upper Bound = Q3 + k * IQR
    """
    """
    .Creates a function that operates on a numeric column (col).
    .Copies the data (so the original is not corrupted).
    .Adds a flag column (e.g. price_outlier_flag) that starts with zero.
    """
    df_in = df_in.copy()
    flag_col = f"{col}_outlier_flag"
    df_in[flag_col] = 0  # Initialize outlier flag column
    
    def _wins(group):
     x = group[col]
     q1 = x.quantile(0.25)
     q3 = x.quantile(0.75)
     iqr = q3 - q1
     low = q1 - k * iqr
     high = q3 + k * iqr
     clipped = x.clip(lower=low, upper=high)
     changed = (clipped != x) & x.notna() # Only flag non-NaN changes. For example, if original is NaN and clipped is NaN, no change.
     
     group[flag_col] = changed.astype("int8")
     group[f"{col}_w"] = clipped
     group[f"{col}_low_iqr"] = low
     group[f"{col}_high_iqr"] = high
     return group
    
    df_out = df_in.groupby(group_cols, group_keys=False).apply(_wins) # apply is used to apply a function along an axis of the DataFrame
    return df_out

for col in num_cols_outlier_policy:
    if col in dfq.columns:
        dfq = winsorize_grouped(dfq, col, group_cols)

for col in num_cols_outlier_policy:
    if f"{col}_outlier_flag" in dfq.columns:
        print(f"{col}: outliers flagged =", int(dfq[f"{col}_outlier_flag"].sum()))



C:\Users\Taha\AppData\Local\Temp\ipykernel_15532\2577900612.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_out = df_in.groupby(group_cols, group_keys=False).apply(_wins) # apply is used to apply a function along an axis of the DataFrame
C:\Users\Taha\AppData\Local\Temp\ipykernel_15532\2577900612.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_out = df_in.groupby(group_cols, group_keys=False).apply(_w

inventory_level: outliers flagged = 0
price: outliers flagged = 0
competitor_price: outliers flagged = 0
units_ordered: outliers flagged = 0
units_sold: outliers flagged = 27


C:\Users\Taha\AppData\Local\Temp\ipykernel_15532\2577900612.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_out = df_in.groupby(group_cols, group_keys=False).apply(_wins) # apply is used to apply a function along an axis of the DataFrame


###    Imputation

In [17]:
# === 7) Imputation for missing values ===
# We'll handle numeric columns separately with simple, transparent rules.

impute_rules = {
    "units_sold": "rolling_mean",        # 7-day rolling mean for long gaps
    "inventory_level": "ffill_then_med", # forward-fill short gaps, median for long gaps
    "price": "ffill_then_mean",          # forward-fill short gaps, mean for long gaps
    "competitor_price": "ffill_then_mean"
}

# Make a copy to avoid messing dfq
df_imp = dfq.copy()

# Helper functions
def impute_units_sold(group, col="units_sold", window=7):
    x = group[col]
    # short gaps interpolate
    x = x.interpolate(limit=3, limit_direction="both")
    # remaining gaps: rolling mean
    if x.isna().any():
        x = x.fillna(x.rolling(window, min_periods=1).mean())
    return x

def impute_inventory(group, col="inventory_level", window=7):
    x = group[col]
    x = x.fillna(method="ffill", limit=2)
    if x.isna().any():
        x = x.fillna(x.rolling(window, min_periods=1).median())
    return x

def impute_price(group, col, window=7):
    x = group[col]
    x = x.fillna(method="ffill", limit=3)
    if x.isna().any():
        x = x.fillna(x.rolling(window, min_periods=1).mean())
    return x

# Apply imputation per (store_id, product_id)
group_cols = ["store_id","product_id"]

for col, rule in impute_rules.items():
    if col not in df_imp.columns:
        continue
    if rule == "rolling_mean":
        df_imp[col+"_imputed"] = (
            df_imp.groupby(group_cols, group_keys=False)
                  .apply(lambda g: impute_units_sold(g, col))
        )
    elif rule == "ffill_then_med":
        df_imp[col+"_imputed"] = (
            df_imp.groupby(group_cols, group_keys=False)
                  .apply(lambda g: impute_inventory(g, col))
        )
    elif rule == "ffill_then_mean":
        df_imp[col+"_imputed"] = (
            df_imp.groupby(group_cols, group_keys=False)
                  .apply(lambda g: impute_price(g, col))
        )

print("✅ Imputation done. Columns with *_imputed added:")
print([c for c in df_imp.columns if c.endswith("_imputed")])


✅ Imputation done. Columns with *_imputed added:
['units_sold_imputed', 'inventory_level_imputed', 'price_imputed', 'competitor_price_imputed']


C:\Users\Taha\AppData\Local\Temp\ipykernel_15532\2013499725.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: impute_units_sold(g, col))
C:\Users\Taha\AppData\Local\Temp\ipykernel_15532\2013499725.py:26: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  x = x.fillna(method="ffill", limit=2)
C:\Users\Taha\AppData\Local\Temp\ipykernel_15532\2013499725.py:52: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the gro

# Cell 1 — pick dataframe + pick target + sort

In [18]:
# 1) pick the most processed dataframe available
for cand in ["df_imp", "dfq", "df"]:
    if cand in globals():
        base_df = globals()[cand].copy()
        print(f"Using dataframe: {cand}")
        break
assert base_df is not None, "No dataframe found (df_imp/dfq/df)."

# 2) ensure datetime + keys as string
base_df["date"] = pd.to_datetime(base_df["date"], errors="coerce")
assert base_df["date"].notna().any(), "No valid dates."

for k in ["store_id", "product_id"]:
    if k in base_df.columns:
        base_df[k] = base_df[k].astype("string")
    else:
        raise ValueError(f"Missing key column: {k}")
    
# 3) choose target (prefer *_imputed)
CANDIDATE_TARGETS = ["inventory_level_imputed","units_sold_imputed","units_sold","units_sold_est","inventory_level"]
TARGET_COL = next((c for c in CANDIDATE_TARGETS if c in base_df.columns), None)
if TARGET_COL is None:
    raise ValueError("No suitable target column found. Add one of: " + ", ".join(CANDIDATE_TARGETS))
print("TARGET_COL =", TARGET_COL)

# 4) sort
base_df = base_df.sort_values(["store_id","product_id","date"]).reset_index(drop=True)
base_df.head(3)

Using dataframe: df_imp
TARGET_COL = inventory_level_imputed


,date,store_id,product_id,inventory_level,price,discount,competitor_price,units_ordered,demand_forecast,holiday_promo,...,units_ordered_low_iqr,units_ordered_high_iqr,units_sold_outlier_flag,units_sold_w,units_sold_low_iqr,units_sold_high_iqr,units_sold_imputed,inventory_level_imputed,price_imputed,competitor_price_imputed
0,2022-01-02,S001,P0001,116,27.95,10,30.89,104,92.94,0,...,-67.0,285.0,0,219.0,-359.625,599.375,219.0,116,27.95,30.89
1,2022-01-03,S001,P0001,154,62.70,20,58.22,189,5.36,0,...,-67.0,285.0,0,151.0,-359.625,599.375,151.0,154,62.70,58.22
2,2022-01-04,S001,P0001,85,77.88,15,75.99,193,52.87,1,...,-67.0,285.0,0,262.0,-359.625,599.375,262.0,85,77.88,75.99


# Cell 2 — define covariates (dynamic + static) with graceful fallback

In [19]:
# dynamic-known candidates (will keep only those existing)
dyn_cov_candidates = [
    "price_imputed","price",
    "competitor_price_imputed","competitor_price",
    "discount",
    "is_holiday_promo","holiday_promo",
    "seasonality",
    "demand_forecast",
    "units_ordered",
    # winsorized versions (optional)
    "price_w","inventory_level_w"
]
DYN_COVS = [c for c in dyn_cov_candidates if c in base_df.columns]

# prefer the _imputed versions by aliasing
def alias_pref(df, preferred, fallback, new_name):
    if preferred in df.columns:
        df[new_name] = df[preferred]
        return new_name
    if fallback in df.columns:
        df[new_name] = df[fallback]
        return new_name
    return None

# normalize common covariates to unified names
norm_covs = []
if alias_pref(base_df, "price_imputed","price","price_feature"):          
    norm_covs.append("price_feature")
if alias_pref(base_df, "competitor_price_imputed","competitor_price","competitor_price_feature"): 
    norm_covs.append("competitor_price_feature")
if "discount" in base_df.columns:
    norm_covs.append("discount")
if "is_holiday_promo" in base_df.columns:
    norm_covs.append("is_holiday_promo")
elif "holiday_promo" in base_df.columns:
    norm_covs.append("holiday_promo")
if "seasonality" in base_df.columns:     
    norm_covs.append("seasonality")
if "demand_forecast" in base_df.columns:  
    norm_covs.append("demand_forecast")
if "units_ordered" in base_df.columns:   
    norm_covs.append("units_ordered")

# static covariates (optional)
STATIC_COVS = [c for c in ["category","region"] if c in base_df.columns]

print("Dynamic covariates (normalized):", norm_covs)
print("Static covariates:", STATIC_COVS)


Dynamic covariates (normalized): ['price_feature', 'competitor_price_feature', 'discount', 'holiday_promo', 'seasonality', 'demand_forecast', 'units_ordered']
Static covariates: ['category', 'region']


# Cell 3 — enforce daily continuity (fill date gaps per series)

In [20]:
def ensure_daily_continuity(g: pd.DataFrame) -> pd.DataFrame:
    g = g.sort_values("date")
    full_idx = pd.date_range(g["date"].min(), g["date"].max(), freq="D")
    g = g.set_index("date").reindex(full_idx).rename_axis("date").reset_index()
    # fill keys/statics
    for k in ["store_id","product_id"]:
        g[k] = g[k].ffill().bfill()
    for c in STATIC_COVS:
        g[c] = g[c].ffill().bfill()
    return g

df_daily = base_df.groupby(["store_id","product_id"], group_keys=False).apply(ensure_daily_continuity).reset_index(drop=True)

# fill covariates softly (only covs, NOT the target)
df_daily = df_daily.sort_values(["store_id","product_id","date"]).reset_index(drop=True)
for c in norm_covs:
    df_daily[c] = (
        df_daily
        .groupby(["store_id","product_id"])[c]
        .transform(lambda s: s.ffill().bfill())
    )


print("Daily continuity enforced. Shape:", df_daily.shape)
df_daily.head(3)


Daily continuity enforced. Shape: (73000, 49)


C:\Users\Taha\AppData\Local\Temp\ipykernel_15532\2008631175.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_daily = base_df.groupby(["store_id","product_id"], group_keys=False).apply(ensure_daily_continuity).reset_index(drop=True)


,date,store_id,product_id,inventory_level,price,discount,competitor_price,units_ordered,demand_forecast,holiday_promo,...,units_sold_outlier_flag,units_sold_w,units_sold_low_iqr,units_sold_high_iqr,units_sold_imputed,inventory_level_imputed,price_imputed,competitor_price_imputed,price_feature,competitor_price_feature
0,2022-01-02,S001,P0001,116,27.95,10,30.89,104,92.94,0,...,0,219.0,-359.625,599.375,219.0,116,27.95,30.89,27.95,30.89
1,2022-01-03,S001,P0001,154,62.70,20,58.22,189,5.36,0,...,0,151.0,-359.625,599.375,151.0,154,62.70,58.22,62.70,58.22
2,2022-01-04,S001,P0001,85,77.88,15,75.99,193,52.87,1,...,0,262.0,-359.625,599.375,262.0,85,77.88,75.99,77.88,75.99


# Cell 4 — build series_id and long-format (date, series_id, target, covariates…)

In [21]:
df_ready = df_daily.copy()
df_ready["series_id"] = df_ready["store_id"].astype(str) + "_" + df_ready["product_id"].astype(str)

cols_model = ["date","series_id", TARGET_COL] + norm_covs
df_ready = df_ready[cols_model].rename(columns={TARGET_COL: "target"})
print("Model columns:", df_ready.columns.tolist())
print("Series:", df_ready["series_id"].nunique(), "| Date range:", df_ready["date"].min(), "→", df_ready["date"].max())
df_ready.head(3)


Model columns: ['date', 'series_id', 'target', 'price_feature', 'competitor_price_feature', 'discount', 'holiday_promo', 'seasonality', 'demand_forecast', 'units_ordered']
Series: 100 | Date range: 2022-01-02 00:00:00 → 2024-01-01 00:00:00


,date,series_id,target,price_feature,competitor_price_feature,discount,holiday_promo,seasonality,demand_forecast,units_ordered
0,2022-01-02,S001_P0001,116,27.95,30.89,10,0,NaN,92.94,104
1,2022-01-03,S001_P0001,154,62.70,58.22,20,0,NaN,5.36,189
2,2022-01-04,S001_P0001,85,77.88,75.99,15,1,NaN,52.87,193


In [23]:
CONTEXT = 512 # input sequence length. it means how many past days to look at
HORIZON = 7 # forecast horizon. it means how many future days to predict
N_FOLDS = 4 # number of folds for cross-validation

df = df_ready.copy()
# Remove 'inplace=True' and rely on the assignment
df = df.drop(columns=['seasonality']) # because it has too many missing values and is not very useful for forecasting we drop it

panel = df.copy()
panel["target_raw"] = panel["target"]
for col in panel.columns:
    print(col)

date
series_id
target
price_feature
competitor_price_feature
discount
holiday_promo
demand_forecast
units_ordered
target_raw


In [24]:
EPS = 1e-8

def _align_torch(y, yhat, strict=False):
    if not torch.is_tensor(y):    y = torch.tensor(y, dtype=torch.float32)
    if not torch.is_tensor(yhat): yhat = torch.tensor(yhat, dtype=torch.float32)
    if y.shape != yhat.shape:
        m = min(y.shape[0], yhat.shape[0])
        if strict:
            raise ValueError(f"Shape mismatch y={y.shape} yhat={yhat.shape}")
        y, yhat = y[:m], yhat[:m]
    return y, yhat

def mae(y, yhat, strict=False):
    y, yhat = _align_torch(y, yhat, strict)
    return torch.mean(torch.abs(y - yhat))

def rmse(y, yhat, strict=False):
    y, yhat = _align_torch(y, yhat, strict)
    return torch.sqrt(torch.mean((y - yhat) ** 2))

def smape(y, yhat, strict=False, eps=EPS):
    y, yhat = _align_torch(y, yhat, strict)
    return 200.0 * torch.mean(torch.abs(y - yhat) / (torch.abs(y) + torch.abs(yhat) + eps))

def wape(y, yhat, strict=False, eps=EPS):
    y, yhat = _align_torch(y, yhat, strict)
    return 100.0 * torch.sum(torch.abs(y - yhat)) / (torch.sum(torch.abs(y)) + eps)

In [25]:
all_dates = np.sort(panel["date"].unique())
assert len(all_dates) > (N_FOLDS + 1) * HORIZON

fold_cutoffs = []
for k in range(N_FOLDS):
    train_end_idx = len(all_dates) - (N_FOLDS - k) * HORIZON
    cutoff = all_dates[train_end_idx - 1]   
    fold_cutoffs.append(pd.Timestamp(cutoff))

fold_cutoffs

[Timestamp('2023-12-04 00:00:00'),
 Timestamp('2023-12-11 00:00:00'),
 Timestamp('2023-12-18 00:00:00'),
 Timestamp('2023-12-25 00:00:00')]

In [26]:
# cutoff is the last date of the training period
# context is how many past days to look at
#horizon is how many future days to predict
def get_hist_and_truth(sdf, cutoff, context=CONTEXT, horizon=HORIZON):
    sdf = sdf.sort_values("date")
    train_mask = sdf["date"] <= cutoff
    #pd.TimeDelta means a duration, the difference between two dates or times.
    # For example, pd.Timedelta(days=5) represents a duration of 5 days.
    val_mask = (sdf["date"]>= cutoff) & (sdf["date"] <= cutoff + pd.Timedelta(days=horizon))

    train_df = sdf.loc[train_mask] # .loc means to access a group of rows and columns by labels or a boolean array
    val_df = sdf.loc[val_mask]

    if len(val_df) < horizon or len(train_df) == 0:
        return [], [], False
    
    y_hist = train_df["target"].astype(float).values[-context:].tolist()

    y_true = val_df["target_raw"].astype(float).values
    if np.isnan(y_true).any():
        return [], [], False

    return y_hist, y_true.tolist(), True

In [27]:
def mean7_forecast(history, horizon=HORIZON):
    if not history:
        return np.zeros(horizon)
    k = min(7, len(history))
    return np.full(horizon, float(np.mean(history[-k:])))

try:
    TFM = timesfm.TimesFM_2p5_200M_torch.from_pretrained("google/timesfm-2.5-200m-pytorch")
    TFM.compile(ForecastConfig(max_context=CONTEXT, max_horizon=HORIZON))
    TFM_OK = True
except Exception as e:
    print("[Warn] TimesFM unavailable → fallback to baseline only.", e)
    TFM = None
    TFM_OK = False

def forecast_batch(hist_list, horizon=HORIZON, batch=64, use_timesfm=True):
    preds = []
    if use_timesfm and TFM_OK:
        for i in range(0, len(hist_list), batch):
            chunk = hist_list[i:i+batch]
            pf, _ = TFM.forecast(horizon=horizon, inputs=[list(map(float, seq)) for seq in chunk])
            preds.extend([np.asarray(x, float) for x in pf])
        return preds
    # fallback: baseline
    return [mean7_forecast(seq, horizon=horizon) for seq in hist_list]

In [28]:
def run_backtest(panel, cutoffs, use_timesfm=True):
    rows = []
    for fold_idx, cutoff in enumerate(cutoffs, 1):
        sids, Hlist, Ytrue = [], [], []
        for sid, sdf in panel.groupby("series_id"):
            y_hist, y_true, ok = get_hist_and_truth(sdf, cutoff)
            if not ok or len(y_hist)==0:
                continue
            sids.append(sid); Hlist.append(y_hist); Ytrue.append(np.asarray(y_true, float))

        if not sids:
            print(f"[fold {fold_idx}] no evaluable series; skip.")
            continue

        # TimesFM یا fallback
        Yhat_main = forecast_batch(Hlist, horizon=HORIZON, use_timesfm=use_timesfm)
        # baseline برای مقایسه
        Yhat_base = [mean7_forecast(h) for h in Hlist]

        # ذخیره متریک‌ها
        for sid, y, yhat_m, yhat_b in zip(sids, Ytrue, Yhat_main, Yhat_base):
            rows.append({
                "fold": fold_idx,
                "cutoff": cutoff,
                "series_id": sid,
                "model": "TimesFM" if (use_timesfm and TFM_OK) else "Mean7",
                "sMAPE": smape(y, yhat_m),
                "MAE":   mae(y, yhat_m),
                "RMSE":  rmse(y, yhat_m),
                "sMAPE_baseline": smape(y, yhat_b),
                "MAE_baseline":   mae(y, yhat_b),
                "RMSE_baseline":  rmse(y, yhat_b),
            })

        fold_mean = np.mean([r["sMAPE"] for r in rows if r["fold"]==fold_idx])
        print(f"[fold {fold_idx}] series={len(sids)} | sMAPE({('TimesFM' if (use_timesfm and TFM_OK) else 'Mean7')}): {fold_mean:.2f}")

    return pd.DataFrame(rows)

metrics_df = run_backtest(panel, fold_cutoffs, use_timesfm=True)
metrics_df.to_csv("backtest_metrics.csv", index=False)
metrics_df.groupby("fold")[["sMAPE","MAE","RMSE","sMAPE_baseline","MAE_baseline","RMSE_baseline"]].mean()

[fold 1] series=100 | sMAPE(TimesFM): 46.02
[fold 2] series=100 | sMAPE(TimesFM): 44.82
[fold 3] series=100 | sMAPE(TimesFM): 45.28
[fold 4] series=100 | sMAPE(TimesFM): 44.64


,sMAPE,MAE,RMSE,sMAPE_baseline,MAE_baseline,RMSE_baseline
fold,,,,,,
1,46.024238,112.636338,128.663047,46.690474,114.94124,133.640371
2,44.824922,114.002109,128.995371,46.732314,118.197354,136.004053
3,45.27834,113.683564,129.842012,46.599116,117.980225,135.171504
4,44.644346,110.18877,125.989873,46.233506,115.139365,134.374121
